# Bibliotecas

## Instalação



* Modelo de PLN Spacy para o idioma Português

In [ ]:
!python -m spacy download pt_core_news_sm

* Biblioteca para Manipulação Planilhas de Excel

In [ ]:
!pip install openpyxl

## Importação

In [107]:
# Manipulação df
import pandas as pd
import numpy as np

# Manipulação texto
import re
import unicodedata

# Contagem de tokens
from collections import Counter

# Modelo Manipulação PLN
import nltk
from nltk.tokenize import word_tokenize

# Modelo PLN
import spacy
from spacy.lang.pt import Portuguese

# Manipulação de Arquivos no Colab
from google.colab import files



# Upload

In [ ]:
df = files.upload()

# Fluxo de Tratamento (EXEC)

In [145]:
df = pd.read_excel('MOTIVOS.xlsx')

* Limpeza de texto com regex:

  - No sistema, o campo observações possui escrita livre. Portanto, não existe nenhum para padrão que possa ser filtrado.

  - Foi observado que, geralmente, o principal motivo é descrito nas primeiras duas frases.

  - Dentro do que foi observado, utilizamos regex para realizar a primeira etapa limpeza e padronização do texto.

In [146]:
def Limpeza_Regex(texto):
  # tira acentos
  texto = unicodedata.normalize('NFD', texto)
  texto = texto.encode('ascii', 'ignore').decode('utf-8', 'ignore')

  # lower case
  texto = texto.lower()

  # remove todos os asteriscos
  texto = re.sub(r'\*+', '', texto)

  # remove todos os '>'
  texto = re.sub(r'\>+', '', texto)

  # remove quebras de linha do início
  texto = re.sub(r'^(\s*(_x000d_|\n))+', '', texto)

    # tratamento especial para o padrão "_x000D_\n_x000D_\n"
  if re.search(r'_x000d_\s*_x000d_\s*', texto):
      # remove o "_x000d_" e espaços extras
      texto = re.sub(r'_x000d_\s*_x000d_\s*', ' ', texto)
      # mantém só até a primeira quebra de linha depois disso
      texto = texto.split('\n')[0]
  else:
      # separa por quebras de linha normalmente
      texto = re.split(r'\n', texto, maxsplit=2)
      texto = '\n'.join(texto[:2])

  # remove códigos tipo _x000d_
  texto = re.sub(r'_x000d_', '', texto)

  # remove quebras de linha no início e no fim
  texto = texto.strip()

  # remove espaços extras
  texto = re.sub(r'\s+', ' ', texto).strip()

  return texto


In [147]:
df['Limpeza_Regex'] = df['obs'].fillna('').apply(lambda x: Limpeza_Regex(x))

* Stop Words

  -   Objeto com lista de que não possuem significado na frase (stop_words)


In [148]:
stop_words = spacy.lang.pt.stop_words.STOP_WORDS

In [149]:
def no_stopwords(texto):
  doc = nlp_pt(texto)
  return " ".join(x.text for x in doc if x.text not in stop_words)

* Lemmatization

  - Retorna as palavras a sua forma base, respeitando gramática e contexto

In [150]:
# carrega modelo pré treinado Spacy para Português
nlp_pt = spacy.load('pt_core_news_sm')

In [151]:
def lemmatization(texto):
  doc = nlp_pt(texto)
  return ' '.join([token.lemma_ for token in doc])

* Applying

In [152]:
df['no_stopwords'] = df['Limpeza_Regex'].apply( lambda x: no_stopwords(x))

In [153]:
df['lemmatization'] = df['no_stopwords'].apply( lambda x: lemmatization(x))

* Segunda limpeza

  - tirando palavras frequentes que não agregam significado, acentos e excesso de espaços.

In [154]:
def Limpeza_Regex2(texto):

  # excluir palavra enviar
  texto = re.sub(r'enviar', '', texto)

  # excluir word 'anexar'
  texto = re.sub(r'anexar', '', texto)

  # excluir word 'prezar'
  texto = re.sub(r'prezar', '', texto)

  # excluir simbolo '/'
  texto = re.sub(r'/', '', texto)

  # excluir simbolo ':'
  texto = re.sub(r':', '', texto)

  # excluir simbolo ','
  texto = re.sub(r',', '', texto)

  # remover espaços duplos ou mais
  texto = re.sub(r'\s{2,}', ' ', texto)

  # substituir '-'
  texto = re.sub(r'-', ' ', texto)

  return texto

In [155]:
df['Limpeza_Regex2'] = df['lemmatization'].apply( lambda x: Limpeza_Regex2(x))

## Estatísticas de texto

* Frequencia de frase texto:

  - Quantas vezes repetiu texto inteiro com o mesmo valor

### Frequencia de tokens com N-Grams

* Processamento de tokens

In [156]:
words = " ".join(df['Limpeza_Regex2'])

* Tokenizador pré treinado do nltk

In [ ]:
nltk.download('punkt_tab')

* Separa o texto em tokens

In [158]:
tokens = word_tokenize(words)

* Unigrams

In [159]:
unigrams = nltk.ngrams(tokens, 1)

In [ ]:
print(Counter(unigrams).most_common(50))

* Bigrams

In [161]:
bigrams = nltk.ngrams(tokens, 2)

In [ ]:
print(Counter(bigrams).most_common(50))

* Concatenar frases:

  - Concatenar frase para valer como tokens únicos

In [163]:
def frases(texto):


  # frase 1
  texto = re.sub(r'alteracao valor', 'alteracao_valor', texto)
  # frase 2
  texto = re.sub(r'confirmar valor', 'confirmar_valor', texto)
  # frase 3
  texto = re.sub(r'documento identificacao', 'documento_identificacao', texto)
  # frase 4
  texto = re.sub(r'3 ultimo extrato', '3_ultimo_extrato', texto)
  # frase 5
  texto = re.sub(r'video ok', '', texto)
  # frase 6
  texto = re.sub(r'selfie legivel', '', texto)

  return texto

In [164]:
df['Limpeza_Regex2'] = df['Limpeza_Regex2'].apply( lambda x: frases(x))

* Listar Palavras Chave para cada Tópico

In [165]:
palavras_topicos = {

    'SELFIE': ['selfie'],
    'ALTERAÇÃO DE VALORES': ['alteracao_valor','confirmar_valor','alteracao', 'alteracoes'],
    'VÍDEO/ÁUDIO PENDENTE': ['script','inaudivel','audio','gravacao','video'],
    'DOCUMENTOS': ['documento_identificacao','3_ultimo_extrato','contrato','extrato','comprovante','documento','detalhamento','print','copia','autorizar','pdf','pdfer','legivel','extrato']}


* Classificação de texto simples baseada em Palavras Chave

In [166]:
def atribuir_topico_com_palavras_chave(texto, palavras_topicos):
    for i, (topico, palavras) in enumerate(palavras_topicos.items()):
        if any(palavra in texto for palavra in palavras):
            return topico  # Retorna o tópico
    return  'DOCUMENTOS'

In [167]:
# Criar coluna de rótulo
df["rótulo_tópico"] = df["Limpeza_Regex2"].apply(lambda x: atribuir_topico_com_palavras_chave(x, palavras_topicos))

* DF para Excel

In [168]:
df.to_excel('Pendencias.xlsx', index=False)

* Download

In [169]:
files.download('Pendencias.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>